In [ ]:
from sklearn import preprocessing
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas
from sklearn.base import TransformerMixin
from collections import Counter

## Scaling

#### StandardScaler

In [ ]:
X_train = np.array([[ 1., -1.,  2.],
[ 2.,  0.,  0.],
[ 0.,  1., -1.]])

Давайте отмасштабируем каждый признак

In [ ]:
X_scaled = preprocessing.scale(X_train)

In [ ]:
X_scaled 

**Что мы ожидаем от этого кода?**

In [ ]:
print(X_scaled.mean(axis=0))
print(X_scaled.std(axis=0))

In [ ]:
scaler = preprocessing.StandardScaler().fit(X_train)
scaler

Параметры преобразования

In [ ]:
scaler.mean_

In [ ]:
scaler.scale_

In [ ]:
X_train

In [ ]:
scaler.transform(X_train)

Перепроверим свойства преобразованных данных

In [ ]:
print(scaler.transform(X_train).mean(axis=0))
print(scaler.transform(X_train).std(axis=0))

In [ ]:
scaler = preprocessing.StandardScaler(with_std=False).fit(X_train)
print(scaler.transform(X_train).mean(axis=0))
print(scaler.transform(X_train).std(axis=0))

#### MinMaxScaler

Масштабировать признаки можно по-разному, в прошлом варианте мы делали значения несмещёнными и с единичной дисперсией, теперь мы будем проектировать значения на заданный отрезок

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()
min_max_scaler

In [ ]:
min_max_scaler.fit(X_train)

Параметры масшбирования немного другие

In [ ]:
min_max_scaler.scale_

In [ ]:
min_max_scaler.min_  

In [ ]:
X_scaled = min_max_scaler.fit_transform(X_train)
X_scaled

Проверяем корректность преобразования

In [ ]:
print(X_scaled.min(axis=0))
print(X_scaled.max(axis=0))

#### MaxAbsScaler

В данном случае нормируем столбцы на наибольшее значение по модулю

In [ ]:
max_abs_scaler = preprocessing.MaxAbsScaler()
max_abs_scaler

In [ ]:
max_abs_scaler.fit(X_train)

In [ ]:
max_abs_scaler.scale_

In [ ]:
X_scaled = max_abs_scaler.fit_transform(X_train)
X_scaled

Минимум и максимум будут из отрезка [-1, 1]

In [ ]:
print(X_scaled.min(axis=0))
print(X_scaled.max(axis=0))

In [ ]:
print(np.abs(X_scaled).min(axis=0))
print(np.abs(X_scaled).max(axis=0))

## Пример использования на датасете

Возьмём стандартный набор данных

In [ ]:
data = load_breast_cancer()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.8, random_state=42)

Обучим  kNN

In [ ]:
algo = KNeighborsClassifier().fit(X_train, y_train)
accuracy_score(algo.predict(X_test), y_test)

По умолчанию kNN использует $l_2$ метрику, поэтому разные мастабы признаков могут негативно сказаться на качестве, давайте попробуем исправить эту проблему

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(preprocessing.scale(data.data), data.target, test_size=0.8, random_state=42)

In [ ]:
algo = KNeighborsClassifier().fit(X_train, y_train)
accuracy_score(algo.predict(X_test), y_test)

**Почему так не корректно проверять?**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.8, random_state=42)

In [ ]:
scaler = preprocessing.StandardScaler().fit(X_train)
algo = KNeighborsClassifier().fit(scaler.transform(X_train), y_train)
accuracy_score(algo.predict(scaler.transform(X_test)), y_test)

Эта оценка более корректна, но лучше использовать kFold кроссвалидацию

Как корректно запустить с cross_val_score?

In [ ]:
cross_val_score(KNeighborsClassifier(), preprocessing.scale(data.data), data.target, cv=3).mean()

Такой вариант некорректен

Для комбинации нескольких шагов обработки данных удобно пользоваться Pipeline

In [ ]:
algo = make_pipeline(preprocessing.StandardScaler(), KNeighborsClassifier())
algo

In [ ]:
cross_val_score(algo, data.data, data.target, cv=3).mean()

# Normalization

Делать масштабирование можно не только в рамках одного признака, но и в рамках одного объекта

Normalizer приводит признаки каждого объекта к единичной норме по какой-то метрике, т.е. нормирует по строкам

In [ ]:
X_train = np.array([[ 1., -1.,  2.],
[ 2.,  0.,  0.],
[ 0.,  1., -1.]])

In [ ]:
normalizer = preprocessing.Normalizer().fit(X_train)
normalizer

In [ ]:
X_scaled = normalizer.transform(X_train)

In [ ]:
X_scaled 

**Какие значения мы можем гарантировать, а какие нет?**

In [ ]:
print(X_scaled.mean(axis=1))
print(X_scaled.std(axis=1))
print((X_scaled ** 2).sum(axis=1))

В pipeline можно комбинировать много шагов, необязательно два

In [ ]:
cross_val_score(
    make_pipeline(preprocessing.StandardScaler(), preprocessing.Normalizer(), KNeighborsClassifier()), 
    data.data, 
    data.target, 
    cv=3
).mean()

# Binarization

In [ ]:
X_train = np.array([[ 1., -1.,  2.],
[ 2.,  0.,  0.],
[ 0.,  1., -1.]])

In [ ]:
binarizer = preprocessing.Binarizer(threshold=0.).fit(X_train)
binarizer

In [ ]:
binarizer.transform(X_train)

## Imputation of missing values

Иногда в данных присутсвуют пропуски, а для корректной работы методов их быть не должно. Sklearn предоставляет стандартные методы обработки пропусков.

In [ ]:
X_train = [
    [1, 2], 
    [np.nan, 3], 
    [7, 6]
]

In [ ]:
imputer = preprocessing.Imputer(missing_values='NaN', strategy='mean', axis=0).fit(X_train)
imputer

In [ ]:
print(imputer.transform(X_train)) 

In [ ]:
X_test = [
    [np.nan, 2], 
    [6, np.nan], 
    [7, 6]
]

In [ ]:
print(imputer.transform(X_test)) 

## Encoding categorical features

## One hot encoding

![](one_hot_encoding.png)

In [ ]:
X_train = np.array([
    [0, 0, 3], 
    [1, 1, 0], 
    [0, 2, 1], 
    [1, 0, 2]
])

In [ ]:
encoder = preprocessing.OneHotEncoder().fit(X_train)
encoder

In [ ]:
encoder.transform([[0, 1, 3]])

In [ ]:
encoder.transform([[0, 1, 3]]).todense()

Можем явно увеличивать количество классов

In [ ]:
encoder = preprocessing.OneHotEncoder(n_values=[3, 2, 4]).fit(X_train)
encoder

In [ ]:
encoder.transform([[0, 1, 3]]).todense()

Но уменьшать не можем

In [ ]:
encoder = preprocessing.OneHotEncoder(n_values=[2, 2, 3]).fit(X_train)
encoder

Пример использования

In [ ]:
df = pandas.read_csv('mushrooms.csv', header=None)

In [ ]:
df.head()

In [ ]:
X, y = np.array(df.loc[:, 1:]), np.array(df.loc[:, 0])

In [ ]:
X

OneHotEncoder требует, чтобы метки были числами

In [ ]:
encoder = preprocessing.OneHotEncoder().fit(X)
encoder

Поэтому сначала нужно преобразовать данные

In [ ]:
label_encoder = preprocessing.LabelEncoder()
label_encoder

In [ ]:
for i in range(X.shape[1]):
    X[:, i] = label_encoder.fit_transform(X[:, i])
X

In [ ]:
encoder = preprocessing.OneHotEncoder().fit(X)
encoder

In [ ]:
X = encoder.fit_transform(X).todense()
X

In [ ]:
y = np.equal(y, 'p').astype(int)
y

In [ ]:
cross_val_score(LogisticRegression(), X, y, cv=3).mean()

## Mean encoding

![](mean_encoding.png)

In [ ]:
X, y = np.array(df.loc[:, 1:]), np.array(df.loc[:, 0])
y = np.equal(y, 'p').astype(int)

for i in range(X.shape[1]):
    le = label_encoder.fit(X[:, i])
    X[:, i] = le.transform(X[:, i])
    for j in range(len(le.classes_)):
        indices = X[:, i] == j
        X[indices, i] = y[indices].mean()

In [ ]:
cross_val_score(LogisticRegression(), X, y, cv=3).mean()

Но, как мы помним, это некорректная оценка

Давайте честно оценим качество

In [ ]:
X, y = np.array(df.loc[:, 1:]), np.array(df.loc[:, 0])
y = np.equal(y, 'p').astype(int)
for i in range(X.shape[1]):
    X[:, i] = label_encoder.fit_transform(X[:, i])

In [ ]:
cross_val_score(
    make_pipeline(
        preprocessing.OneHotEncoder(),
        LogisticRegression()
    ),
    X, 
    y,
    cv=10,
).mean()

In [ ]:
cross_val_score(
    make_pipeline(
        preprocessing.OneHotEncoder(),
        LogisticRegression()
    ),
    X, 
    y,
    cv=3
).mean()

Мы не указали поведение encoder-a на ранее не наблюдаемых значениях признака, поэтому получаем ошибку, если указать поведение явно, то ошибки не будет

**Почему при cv=10 ошибки не было?**

In [ ]:
cross_val_score(
    make_pipeline(
        preprocessing.OneHotEncoder(handle_unknown='ignore'),
        LogisticRegression()
    ),
    X, 
    y,
    cv=3
).mean()

Чтобы сделать pipeline напишем собственный трансформер

Чтобы не писать лишних методов наседуемся от базового класса TransformerMixin - теперь не нужно реализовывать fit_transform

In [ ]:
class MeanTransformer(TransformerMixin):
    
    def fit(self, X, y):
        self.cnt = Counter()
        for i in range(X.shape[1]):
            for j in range(np.max(X[:, i])):
                indices = X[:, i] == j
                if np.sum(indices) > 0:
                    val = y[indices].mean()
                else:
                    val = y.mean()
                self.cnt[(i, j)] = val
                
        return self
    
    def transform(self, X):
        X_new = np.copy(X)
        for i in range(X.shape[1]): 
            for j in range(np.max(X[:, i])):
                indices = X[:, i] == j
                if np.sum(indices) > 0:
                    X_new[indices, i] = self.cnt[(i, j)]
        return X_new


In [ ]:
X, y = np.array(df.loc[:, 1:]), np.array(df.loc[:, 0])
y = np.equal(y, 'p').astype(int)
for i in range(X.shape[1]):
    X[:, i] = label_encoder.fit_transform(X[:, i])

In [ ]:
cross_val_score(
    make_pipeline(
        MeanTransformer(),
        LogisticRegression()
    ),
    X, 
    y,
    cv=3
).mean()

In [ ]:
cross_val_score(
    make_pipeline(
        MeanTransformer(),
        LogisticRegression()
    ),
    X, 
    y,
    cv=10
).mean()

Как видите, качество заметно ниже



### Напишите MeanTransformer, который бы при обучении считал счётчики не по всей обучающей выборке, а только по предыдущим объектам - придётся написать свой метод fit_transform